In [1]:
import sys
sys.path.append('../')
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import json
import urllib
import time 
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
from glob import glob
from collections import ChainMap
from tqdm.notebook import tqdm as tqdm_notebook
from datetime import datetime 
from selenium.webdriver.common.keys import Keys
from config import chromeSetup,get_selected_cols_df,ensure_dir,get_Drivers,encode_search_word,enter_sys_value,update_iod_tms_with_now,transform

In [2]:

from config import Marco

marco = Marco()
marco.colsSend2Carrier

['Sales',
 'Load ID',
 'Stop Seq',
 'Carrier Code',
 'Carrier Name',
 'Service Type',
 'Alert Type',
 'Ship Date',
 'ETA',
 'From Address',
 'To Address',
 'Customer Name',
 'GERP Order No',
 'Pick Order No',
 'Pro No']

In [3]:
sup = pd.read_excel(glob("table_result\\result\\utp2.xlsx")[-1])
sup.head()
for c in sup.columns:
    sup[c] = sup[c].astype(str)

sub_dit =  { x: y for i,(x,y) in sup.iterrows() }


In [4]:

chrome_options = Options()
chrome_options.add_argument("--headless")

browers = webdriver.Chrome("C:\\Users\\marco.wang\\Downloads\\chromedriver_win32\\chromedriver")

CARRIER_CODE = ['UTP2',
               ]
FILE_PATH = glob(".\data\*.csv")[-1]
df_origin = pd.read_csv(FILE_PATH)
df = df_origin[df_origin['Carrier Code'].isin(CARRIER_CODE)]



from selenium import webdriver
import time 

 
    




In [5]:
df['Pro No'] = df['Pro No'].apply(lambda x : sub_dit[x] if x in sub_dit else x )

C:\Users\marco.wang\Miniconda3\envs\pantos\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
browers.get("https://client.freightapp.com/tms/template/orderTracking.html")
dit = {}

In [7]:
idx = browers.find_element_by_xpath('//*[@id="details_section"]/div/div').text.split("\n")

In [8]:

for load in tqdm_notebook(df['Pro No']):
    search = browers.find_element_by_xpath('//*[@id="tracking_no"]')
    search.clear()
    time.sleep(0.1)
    search.send_keys(load)

    # search.clear()
    search.send_keys(Keys.ENTER)
    time.sleep(1.2)
    try:
        log_list = browers.find_element_by_xpath('//*[@id="log_list"]')
        state = log_list.text.split("\n")
        details = browers.find_element_by_xpath('//*[@id="details_section"]/div/div').text.split("\n")
        idx = details[1].index('Delivery Appointment')
        dit[load] = {
            "update date:": state[1],
            "status" : state[2],
            "appt":details[1][idx:]
        }
    except:
        print(load)


50102356
50102404
50102402
50102401
50102428
50102427
50102426
50102425



In [9]:
timestamp = str(datetime.now())        
today_date = timestamp[:10]
print(today_date)
timestamp = "".join([d for d  in timestamp if d.isdigit()])
ensure_dir(os.path.join("table_result",today_date))
df = df[marco.colsSend2Carrier]


2021-06-07


In [10]:
df['update date'] = df['Pro No'].apply(lambda x : dit[x]['update date:'] if dit.get(x) else "NA")
df['status'] = df['Pro No'].apply(lambda x : " ".join(dit[x]['status'].split()[2:]) if dit.get(x) else "NA")
df['appt time'] = df['Pro No'].apply(lambda x : dit[x]['appt'] if dit.get(x) else "NA")
df['delivered_completed'] = df['status'].apply(lambda x : "Yes" if "Delivery Completed" in x else "No")
df['Stop Seq'] = df['From Address'].apply(lambda x : x[-12:][:2])
df.rename(columns= {
    'Stop Seq':"Ship State"
},inplace = True)

df = df.sort_values("Ship State")
df.to_excel(os.path.join("table_result",today_date,f"report_UTP2_{timestamp[8:12]}.xlsx"),index=False,engine='xlsxwriter')